In [ ]:
!pip install fastdtw

In [ ]:
# Here we have useful import
import os, os.path, requests, zipfile, io
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path

import plotly
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.preprocessing import MinMaxScaler, StandardScaler

from matplotlib.patches import ConnectionPatch
import matplotlib.pyplot as plt
import numpy as np
import scipy.spatial.distance as dist

# Configuring Matplotlib
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
savefig_options = dict(format="png", dpi=300, bbox_inches="tight")

# Computation packages
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw

# Plotting package
import seaborn as sbn

## Set constant variable

In [ ]:
# Google Colab
# LOCAL_PATH = "/content/dataset"
# Local
LOCAL_PATH = "dataset"

DIR_DATA = "dataset/"

# Name: 	TICKER_Year-Month-Day_StartTime_EndTime_message_LEVEL.csv 	

TICKERS = ["AMZN", "AAPL", "GOOG", "INTC", "MSFT"]

START_TIME = "34200000" 
END_TIME = "57600000" 

MESSAGE = ["message", "orderbook"]

YEAR = "2012"
MONTH = "06"
DAY = "21"
DATE = YEAR + "-" + MONTH + "-" + DAY
START_DATE_DT = dt.datetime.strptime(DATE, "%Y-%m-%d")

LEVEL = "1"

MESSAGE_COLUMNS = ["time", "event_type", "order_id", "size", "price", "direction"]
EXECUTED_ORDER_MESSAGES = [4,5] # 4 and 5 refers to the executed trades

# 1 SEC, 10 SEC, 1MIN, 1 HOUR
UNIT_TIME = "60s"

# Set true if you want to normalize the data
NORMALIZE = True

## Check Directory

In [ ]:
# First we create the directory where to save the CSV files
if os.path.exists(LOCAL_PATH):
  print("Directory dataset exists!")
else:
  directory = "dataset"
  os.mkdir(directory) 
  print("Directory '% s' created" % directory)

## Functions

#### Functions to manage the dataset

In [ ]:
def get_dataframe(ticker, msg_ord, msg_ord_col):
  """
  Read Dataset From a csv file
  Args:
    tricker: string, name of the action ("AMZN", "AAPL", "GOOG", "INTC", "MSFT").
    msg_ord: string, it atcs like a flag, the two possible values are "message" and "orderbook".
    msg_ord_col: list of string, all values for the "message" or "orderbook" columns.
  Return:
   df: pandas dataframe.
  """
  FORMAT_FILE = "{}_{}" + "_" + START_TIME + "_" + END_TIME + "_" + msg_ord + "_" +"{}.csv"
  df = pd.read_csv(DIR_DATA + FORMAT_FILE.format(ticker, DATE, LEVEL), names=msg_ord_col)
  return df


def compute_orderbook_columns(level: int):
  """
  Function to compute the orderbook columns name
  Args: 
     level: int.
  Return: 
    orderbook_columns: list of string
  """
  orderbook_columns = []
  for i in range(1, level+1):
    orderbook_columns += ["sell"+str(i), "vsell"+str(i), "buy"+str(i), "vbuy"+str(i)]
  return orderbook_columns


def compute_joined_lob(message_df, orderbook_df):
  """
  Function to join the Message and Orderbook datsets
  Args:
    message_df: pandas dataframe
    orderbook_df: pandas dataframe
  Return:
    joined_lob_df: pandas dataframe
  """
  joined_lob_df = message_df.copy()
  # Create new columns and initialize their values
  joined_lob_df[ORDERBOOK_COLUMNS] = orderbook_df
  # Add date column to the dataframe
  joined_lob_df.insert(0, "date", [START_DATE_DT + dt.timedelta(seconds=i) for i in joined_lob_df["time"]])
  return joined_lob_df



def compute_ohlc_volume(df):
  """
  Function to compute the OHCL dataset template
  Args:
    df: pandas dataframe
  Return:
    df: pandas dataframe
  """
  executed_trades_messages = df[(df["event_type"].isin(EXECUTED_ORDER_MESSAGES))]
  # We want the price in dollar so we divided by 10000
  executed_trades_messages["price"] = executed_trades_messages["price"] / 10000
  executed_trades_messages.index = executed_trades_messages["date"]
  # Compute the OHLC
  df = executed_trades_messages["price"].resample(UNIT_TIME).ohlc()
  # Compute the volume
  df["volume"] = executed_trades_messages["size"].resample(UNIT_TIME).sum()
  # Manage the missing values (NaN) and in the volume where we have missing values we want 0!
  df[["open", "high", "low", "close"]] = df[["open", "high", "low", "close"]].fillna(method="ffill")
  df[["open", "high", "low", "close"]] = df[["open", "high", "low", "close"]].fillna(method="bfill")
  df["volume"] = df["volume"].fillna(0)
  return df



def compute_r(df, k=5):
  """
  Function to compute the R values, using Rolling Mean
  Args:
    df: pandas dataframe
  Return:
    df: pandas dataframe
  """
  # mean k previous values
  m_minus = df["open"].rolling(k).mean() 
  # mean k future values
  m_plus = df["open"].shift(-(k-1)).rolling(k).mean() 
  df["R"] = (m_plus - m_minus) / m_minus
  df["R"] = df["R"].fillna(method="ffill").fillna(method="bfill")
  return df

In [ ]:
def select_date_range(df, start_date, end_date): 
  #greater than the start date and smaller than the end date
  mask = (df['date'] > start_date) & (df['date'] <= end_date)
  df = df.loc[mask]
  return df

### Bollinger Bands

#### Compute Bollinger Bands

In [ ]:
def compute_bollinger_band(data, col_name):
  """
  Function to compute the bollinger bands using a window of 20 days
  Args:
    data: pandas dataframe
    col_name: string, "close" or "volume"
  Return:
    buyers, sellers, upper_band, lower_band, sma: list
  """
  df = data[[col_name]]
  sma = df.rolling(window=20).mean().dropna()
  rstd = df.rolling(window=20).std().dropna()

  upper_band = sma + 2 * rstd
  lower_band = sma - 2 * rstd

  upper_band = upper_band.rename(columns={col_name: 'upper'})
  lower_band = lower_band.rename(columns={col_name: 'lower'})
  bb = df.join(upper_band).join(lower_band)
  bb = bb.dropna()

  buyers = bb[bb[col_name] <= bb['lower']]
  sellers = bb[bb[col_name] >= bb['upper']]

  return buyers, sellers, upper_band, lower_band, sma

#### Plot Bollinger Bands

In [ ]:
def plot_bollinger_band(buyers, sellers, upper_band, lower_band, sma, data, col_name, ticker):
  # pio.templates.default = "plotly_dark"

  fig = go.Figure()
  fig = make_subplots(rows=1,cols=1)
  fig.add_trace(go.Scatter(x=lower_band.index, 
                          y=lower_band['lower'], 
                          name='Lower Band', 
                          line_color='rgba(173,204,255,0.2)'
                          ))
  
  fig.add_trace(go.Scatter(x=upper_band.index, 
                          y=upper_band['upper'], 
                          name='Upper Band', 
                          fill='tonexty', 
                          fillcolor='rgba(173,204,255,0.2)', 
                          line_color='rgba(173,204,255,0.2)'
                          ))
  
  fig.add_trace(go.Scatter(x=data.index, 
                          y=data[col_name], 
                          name=col_name, 
                          line_color='#636EFA'
                          ))
  
  fig.add_trace(go.Scatter(x=sma.index, 
                          y=sma[col_name], 
                          name='SMA', 
                          line_color='#FECB52'
                          ))
  if col_name == "close":
    fig.add_trace(go.Scatter(x=buyers.index, 
                            y=buyers[col_name], 
                            name='Buyers', 
                            mode='markers',
                            marker=dict(
                                color='#00CC96',
                                size=10,
                                )
                            ))
    
    fig.add_trace(go.Scatter(x=sellers.index, 
                            y=sellers[col_name], 
                            name='Sellers', 
                            mode='markers', 
                            marker=dict(
                                color='#EF553B',
                                size=10,
                                )
                            ))
  if col_name == "volume":
    fig.add_trace(go.Scatter(x=sellers.index, 
                        y=sellers[col_name], 
                        name='High Volume', 
                        mode='markers',
                        marker=dict(
                            color='#00CC96',
                            size=10,
                            )
                        ))

  
  # Set title
  fig.update_layout(title_text=ticker+" "+col_name)

  fig.update_yaxes(autorange = True, fixedrange= False)


  fig.show()

### DTW

In [ ]:
def dp(dist_mat):
    """
    Find minimum-cost path through matrix `dist_mat` using dynamic programming.

    The cost of a path is defined as the sum of the matrix entries on that
    path. See the following for details of the algorithm:

    - http://en.wikipedia.org/wiki/Dynamic_time_warping
    - https://www.ee.columbia.edu/~dpwe/resources/matlab/dtw/dp.m

    The notation in the first reference was followed, while Dan Ellis's code
    (second reference) was used to check for correctness. Returns a list of
    path indices and the cost matrix.
    """

    N, M = dist_mat.shape
    
    # Initialize the cost matrix
    cost_mat = np.zeros((N + 1, M + 1))
    for i in range(1, N + 1):
        cost_mat[i, 0] = np.inf
    for i in range(1, M + 1):
        cost_mat[0, i] = np.inf

    # Fill the cost matrix while keeping traceback information
    traceback_mat = np.zeros((N, M))
    for i in range(N):
        for j in range(M):
            penalty = [
                cost_mat[i, j],      # match (0)
                cost_mat[i, j + 1],  # insertion (1)
                cost_mat[i + 1, j]]  # deletion (2)
            i_penalty = np.argmin(penalty)
            cost_mat[i + 1, j + 1] = dist_mat[i, j] + penalty[i_penalty]
            traceback_mat[i, j] = i_penalty

    # Traceback from bottom right
    i = N - 1
    j = M - 1
    path = [(i, j)]
    while i > 0 or j > 0:
        tb_type = traceback_mat[i, j]
        if tb_type == 0:
            # Match
            i = i - 1
            j = j - 1
        elif tb_type == 1:
            # Insertion
            i = i - 1
        elif tb_type == 2:
            # Deletion
            j = j - 1
        path.append((i, j))

    # Strip infinity edges from cost_mat before returning
    cost_mat = cost_mat[1:, 1:]
    return (path[::-1], cost_mat)

In [ ]:
def compute_path_costmat(x, y):    
    N = x.shape[0]
    M = y.shape[0]
    print("Shape:", N, M)
    dist_mat = np.zeros((N, M))
    for i in range(N):
        for j in range(M):
            dist_mat[i, j] = abs(x[i] - y[j])

    # DTW
    path, cost_mat = dp(dist_mat)
    print("Alignment cost: {:.4f}".format(cost_mat[N - 1, M - 1]))
    print("Normalized alignment cost: {:.4f}".format(cost_mat[N - 1, M - 1]/(N + M)))

    return path, cost_mat, dist_mat


### DTW Alternative version

In [ ]:
def compute_euclidean_distance_matrix(x, y) -> np.array:
    """Calculate distance matrix
    This method calcualtes the pairwise Euclidean distance between two sequences.
    The sequences can have different lengths.
    """
    dist = np.zeros((len(y), len(x)))
    for i in range(len(y)):
        for j in range(len(x)):
            dist[i,j] = (x[j]-y[i])**2
    return dist

In [ ]:
def compute_accumulated_cost_matrix(x, y) -> np.array:
    """Compute accumulated cost matrix for warp path using Euclidean distance
    """
    distances = compute_euclidean_distance_matrix(x, y)

    # Initialization
    cost = np.zeros((len(y), len(x)))
    cost[0,0] = distances[0,0]
    
    for i in range(1, len(y)):
        cost[i, 0] = distances[i, 0] + cost[i-1, 0]  
        
    for j in range(1, len(x)):
        cost[0, j] = distances[0, j] + cost[0, j-1]  

    # Accumulated warp path cost
    for i in range(1, len(y)):
        for j in range(1, len(x)):
            cost[i, j] = min(
                cost[i-1, j],    # insertion
                cost[i, j-1],    # deletion
                cost[i-1, j-1]   # match
            ) + distances[i, j] 
            
    return cost

#### Plot DTW

In [ ]:
def plot_dtw(dist_mat, cost_mat, path):
    plt.figure(figsize=(6, 4))
    plt.subplot(121)
    plt.title("Distance matrix")
    plt.imshow(dist_mat, cmap=plt.cm.binary,
            interpolation="nearest", origin="lower")
    plt.subplot(122)
    plt.title("Cost matrix")
    plt.imshow(cost_mat, cmap=plt.cm.binary,
            interpolation="nearest", origin="lower")
    x_path, y_path = zip(*path)
    plt.plot(y_path, x_path)

def plot_dtw_path(x, y, path):
    plt.figure()
    for x_i, y_j in path:
        plt.plot([x_i, y_j], [x[x_i] + 1.5, y[y_j] - 1.5], c="C7")
    plt.plot(np.arange(x.shape[0]), x + 1.5, "-o", c="C3")
    plt.plot(np.arange(y.shape[0]), y - 1.5, "-o", c="C0")
    plt.axis("off")


### Plot DTW Alternative

In [ ]:
def plot_dwt_alternative(cost_matrix, warp_path):

    fig, ax = plt.subplots(figsize=(12, 8))
    ax = sbn.heatmap(cost_matrix, annot=True, square=True, linewidths=0.1, cmap="YlGnBu", ax=ax)
    ax.invert_yaxis()

    # Get the warp path in x and y directions
    path_x = [p[0] for p in warp_path]
    path_y = [p[1] for p in warp_path]

    # Align the path from the center of each cell
    path_xx = [x+0.5 for x in path_x]
    path_yy = [y+0.5 for y in path_y]

    ax.plot(path_xx, path_yy, color='blue', linewidth=3, alpha=0.2)

    fig.savefig("ex1_heatmap.png", **savefig_options)

### Main Function

In [ ]:
# Compute ORDERBOOK Columns
ORDERBOOK_COLUMNS = compute_orderbook_columns(int(LEVEL))

In [ ]:
def see_ticker_stat(ticker):
  # Name: TICKER_Year-Month-Day_StartTime_EndTime_message_LEVEL.csv 	
  NAME_MSG = ticker + "_" + DATE + "_" + START_TIME + "_" + END_TIME + "_" + MESSAGE[0] + "_" + LEVEL + ".cvs"
  NAME_ORD = ticker + "_" + DATE + "_" + START_TIME + "_" + END_TIME + "_" + MESSAGE[1] + "_" + LEVEL + ".cvs"
  path_msg = LOCAL_PATH + "/" + NAME_MSG
  path_ord = LOCAL_PATH + "/" + NAME_ORD

  file_path_msg =  Path(path_msg)
  file_path_ord =  Path(path_ord)

  # Download the dataset if it not exists
  if file_path_msg.exists() and file_path_ord.exists():
    print("Datasets of ", ticker, " exist!")
  else:
    URL = "https://lobsterdata.com/info/sample/LOBSTER_SampleFile_" + ticker + "_" + DATE + "_" + LEVEL + ".zip"
    r = requests.get(URL)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(LOCAL_PATH)


  # Message Dataframe
  message_df = get_dataframe(ticker, MESSAGE[0], MESSAGE_COLUMNS)
  
  # Orderbook Dataframe
  orderbook_df = get_dataframe(ticker, MESSAGE[1], ORDERBOOK_COLUMNS)

  # Joined
  joined_lob_df = compute_joined_lob(message_df, orderbook_df)

  # Date Range
  # df_range_date = select_date_range(joined_lob_df,  "2012-06-21 09:30:00", "2012-06-21 09:35:00")

  # OHLC and Volume
  # df_ohlc = plot_ohlc_volume(df_range_date)
  df_ohlc = compute_ohlc_volume(joined_lob_df)

  # Compute the R-value, we use the Rolling Mean
  df_ohlc = compute_r(df_ohlc)

  # Compute Bollinger Band
  cl_buyers, cl_sellers, cl_upper_band, cl_lower_band, sma_cl = compute_bollinger_band(df_ohlc,"close")
  vl_buyers, vl_sellers, vl_upper_band, vl_lower_band, sma_vl = compute_bollinger_band(df_ohlc,"volume")

  # Plot Bollinger Band
  plot_bollinger_band(cl_buyers, cl_sellers, cl_upper_band, cl_lower_band, sma_cl, df_ohlc, "close", ticker)
  plot_bollinger_band(vl_buyers, vl_sellers, vl_upper_band, vl_lower_band, sma_vl, df_ohlc, "volume", ticker)
  
  # Compute MSI
  # mfi = compute_msi(df_ohlc)
  # df_ohlc["mfi"] = mfi

  # Plot MSI
  # plot_mfi(df_ohlc)

  # DTW
  """
  x = df_ohlc["close"]
  y = df_ohlc["volume"]
  if NORMALIZE:
      scaler = StandardScaler()
      reshape = x.values.reshape(-1, 1)
      x = scaler.fit_transform(reshape)
      reshape = y.values.reshape(-1, 1)
      y = scaler.fit_transform(reshape)
  else:
    # Compute Pearson Correlation Coefficient
    # Note: we compute if only if our data are not normalized
    pcc = np.corrcoef(x,y)
    print("Pearson Correlation Coefficient: ", pcc[0,1])
  path, cost_mat, dist_mat = compute_path_costmat(x, y)
  
  plot_dtw(dist_mat, cost_mat, path)
  plot_dtw_path(x, y, path)
  """

  # DTW alternative
  x = df_ohlc["close"]
  y = df_ohlc["volume"]
  if NORMALIZE:
    scaler = StandardScaler()
    reshape = x.values.reshape(-1,1)
    x = scaler.fit_transform(reshape)
    reshape = y.values.reshape(-1,1)
    y = scaler.fit_transform(reshape)
  # compute the distance
  dtw_distance, warp_path = fastdtw(x, y, dist=euclidean)
  cost_matrix = compute_accumulated_cost_matrix(x, y)
  # Print the values of the DTW distance and warp path
  print("DTW distance: ", dtw_distance)
  print("Warp path: ",  warp_path)
  # Plot
  plot_dwt_alternative(cost_matrix, warp_path)  
  

## Plot

### Amazon

In [ ]:
 # AMZN
see_ticker_stat(TICKERS[0])

#### Considerations

With Normalization, from the cost matrix graph, we can see that there are misalignments that lead the optimal path away from the diagonal. Furthermore, we can observe that the optimal path approaches the diagonal only at the opening and closing of the markets.

### Apple 

In [ ]:
#see_ticker_stat(TICKERS[1])


#### Considerations

With Normalization, from the cost matrix graph, we can see that there are misalignments that lead the optimal path away from the diagonal.

### Google

In [ ]:
#see_ticker_stat(TICKERS[2])


#### Considerations

With Normalization, from the cost matrix graph, we can see that there are misalignments that lead the optimal path away from the diagonal. 

### Intel

In [ ]:
#see_ticker_stat(TICKERS[3])


#### Considerations

With Normalization, from the cost matrix graph, we can see that there are misalignments that lead the optimal path away from the diagonal.

### Microsoft

In [ ]:
#see_ticker_stat(TICKERS[4])


#### Considerations

With Normalization, from the cost matrix graph, we can see that there are misalignments that lead the optimal path away from the diagonal. 

## Table for Pearson Correlation Coefficient

The value will be a number between -1 and 1, where 1 is a perfect positive linear relationship, 0 is no linear relationship, and -1 is a perfect negative linear relationship. Different authors use slightly different interpretations of the coefficients, but they’re generally very similar to the ones below.

| Pearson's r value	| Strength of relationship                  |
|-------------------|-------------------------------------------|
| 0	                | No linear relationship                    |
| 0.1 to 0.3	    | Weak linear relationship                  |
| 0.3 to 0.5	    | Moderate linear relationship              |
| 0.5 to 0.7	    | Strong linear relationship                |
| 0.7 to 1	        | Very strong linear relationship           |
| -0.1 to -0.3	    | Weak negative linear relationship         |
| -0.3 to -0.5	    | Moderate negative linear relationship     |
| -0.5 to -0.7	    | Strong negative linear relationship       |
| -0.7 to -1	    | Very strong negative linear relationship  |

The Pearson Correlation Coefficient for all the stocks of our Dataframe is on the range `[0, -0.3]` this means that we have a weak negative linear relationship as we can see from the table.    